# Data Visualisation in Python

## Import Necessary Libraries

In [ ]:
#! pip install plotly

In [ ]:
#! pip install folium

In [ ]:
# for data 
import pandas as pd
# for scientific computation
import numpy as np

In [ ]:
# for visualisation
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
import plotly.io as pio

In [ ]:
import folium

## Data Preparation

### Read Data

In [ ]:
# read the data
df = pd.read_csv("../../Data/Meteorite_Landings_20240212.csv", sep = ',')

In [ ]:
df.shape

In [ ]:
# get idea of the look
df.sample(10)

### Analytical Exploration

In [ ]:
# get idea of columns and types
df.info()

In [ ]:
df.describe()

In [ ]:
# change the format to avoid scientific notation, e.g. e+04
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [ ]:
df['year'] = df['year'].astype(int)

In [ ]:
# see which are the attribute labels
list(df)

In [ ]:
# to check null values in data
df.isnull().sum()

In [ ]:
# delete rows with unknown year
df.dropna(subset=['year'], inplace=True)

In [ ]:
# replace the missing mass with avarage
mass_mean = df['mass (g)'].mean()
df['mass (g)'].fillna(int(mass_mean), inplace=True)

In [ ]:
# check observation types
nametype_group = df.groupby(by='nametype').size()
nametype_group

In [ ]:
# relic only
dfr = df[df['nametype'] == 'Relict']

In [ ]:
dfr.describe()

In [ ]:
# valid only
dfv = df[df['nametype'] == 'Valid']

In [ ]:
dfv.describe()

In [ ]:
df.recclass.unique().tolist()

In [ ]:
recclass_group = df.groupby(by = 'recclass').size()
recclass_group

### Visual Exploration

In [ ]:
#create histogram to visualize values in dataset
df.hist()

In [ ]:
# plot all observations to discover outliers
df.plot.scatter(x='year', y='mass (g)', c='year', colormap='viridis')

In [ ]:
# z-score filter of outliers: (z-score < 3*std) in a column
from scipy import stats
dfny = df[np.abs(stats.zscore(df['year'])) < 3.5]
dfnm = dfny[np.abs(stats.zscore(df['mass (g)'])) < 3.5]
dfnm.plot.scatter(x='year', y='mass (g)', c='year', colormap='viridis')

In [ ]:
# visualise the features and the response using scatterplots
sns.pairplot(dfnm, x_vars=['year', 'recclass'], y_vars='mass (g)', height=5, aspect=0.8)

In [ ]:
# compare in box-plot
ax = df.boxplot(by='nametype', column='mass (g)', figsize=(4, 4))
ax.set_xlabel('Class')
ax.set_ylabel('Mass (g)')
plt.suptitle('')
plt.title('')

plt.tight_layout()
plt.show()

In [ ]:
%matplotlib inline
nametype_group.plot.bar()

### Interactive 3D Visualisation with Plotly

In [ ]:
# line chart
fig = px.line(x=[1,2, 3], y=[1, 2, 3]) 
 
# showing the plot
fig.show()

In [ ]:
fig = px.line_3d(x=[1, 2, 3, 4, 5, 6], y=[6, 5, 4, 3, 2, 1], z=[1, 2, 3, 1, 2, 3]) 
fig.show()

In [ ]:
# 3D scatter plot
fig = px.scatter_3d(dfnm, x="year", y="mass (g)", z='recclass', color="year", size='mass (g)', size_max=40, opacity=0.8)
fig.show()

In [ ]:
# bar chart
fig = px.bar(dfnm, x="year", y="mass (g)", color = 'year') 
fig.show()

In [ ]:
# pie chart
x=[[1, 2, 3, 4, 5], ['a', 'b', 'c', 'd', 'e']]

fig = px.pie(x, values = x[0], names = x[1]) 
fig.show()

In [ ]:
import plotly.graph_objects as go 
import numpy as np 
 
# Data to be plotted
x = np.outer(np.linspace(-2, 2, 30), np.ones(30)) 
y = x.copy().T 
z = np.cos(x ** 2 + y ** 2) 
 
# plotting the figure
fig = go.Figure(data=[go.Surface(x=x, y=y, z=z)]) 
 
fig.show()

### Visualise Geodata with Folium

In [ ]:
dfnm.shape

In [ ]:
dfnm.isnull().sum()

In [ ]:
subset = ['reclat','reclong']
dfnm.dropna(subset = subset, inplace=True)

In [ ]:
dfnm.shape

In [ ]:
dfnm.sample(10)

In [ ]:
# build a map
# center = folium.Marker([55.6819, 12.5627], popup="Cphbusiness-Lyngby", tooltip="new building")
fmap = folium.Map(location = [55.6819, 12.5627], zoom_start = 3)

In [ ]:
# add countries
political_countries_url = ("http://geojson.xyz/naturalearth-3.3.0/ne_50m_admin_0_countries.geojson")
folium.GeoJson(political_countries_url).add_to(fmap)

In [ ]:
fmap

In [ ]:
# add markers for the meteorites
for index, row in dfnm.iloc[:100].iterrows():
    marker = ''
    loc = row['reclat'], row['reclong'] 
    pop = str(int(row['year']))
    marker = folium.Marker(location=loc, popup=pop, 
                           tooltip = "Click me!", icon = folium.Icon(color="green", icon='flag'))
    marker.add_to(fmap)

In [ ]:
fmap

In [ ]:
fmap.add_child(folium.LatLngPopup())

In [ ]:
# add one more layer
folium.TileLayer('openstreetmap').add_to(fmap)

In [ ]:
folium.LayerControl().add_to(fmap)

In [ ]:
fmap.save("../data/meteorit.html")

In [ ]:
fmap

## See More
- https://plotly.com
- https://darigak.medium.com/your-guide-to-folium-markers-b9324fc7d65d
- https://deparkes.co.uk/2016/06/10/folium-map-tiles/

### Try various formatting parameters